In [1]:
import pandas as pd
import numpy as np
import re
from hanspell import spell_checker
from konlpy.tag import Mecab
import konlpy

In [8]:
trade_df = pd.read_excel('data/crawling/유통_무역_운송.xls')
edu_df = pd.read_excel('data/crawling/교육업.xls')
cons_df = pd.read_excel('data/crawling/건설업.xls')

In [9]:
trade_df.drop('Unnamed: 0', axis = 1, inplace = True)
edu_df.drop('Unnamed: 0', axis = 1, inplace = True)
cons_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [10]:
trade_df.set_index('회사', inplace = True)
edu_df.set_index('회사', inplace = True)
cons_df.set_index('회사', inplace = True)

In [11]:
trade_df.drop_duplicates(['요약','장점','단점','경영진에게 바라는 점'], inplace = True)
edu_df.drop_duplicates(['요약','장점','단점','경영진에게 바라는 점'], inplace = True)
cons_df.drop_duplicates(['요약','장점','단점','경영진에게 바라는 점'], inplace = True)

In [34]:
cons_df.isnull().sum()

날짜             0
직무             0
요약             0
장점             0
단점             0
경영진에게 바라는 점    0
dtype: int64

In [33]:
# null값 제거
index = cons_df[cons_df['경영진에게 바라는 점'].isnull()].index
cons_df.drop(index, inplace = True)

In [53]:
# 영어와 한글만 남기는 함수 정의
def clean_text(df):
    for col in df.columns[2:]:
        df[col] = df[col].str.replace("[^가-힇]", " ")

In [12]:
# py-hanspell을 이용한 맞춤법 검사 함수 정의
def spelling(df):
    # [요약]부터 [경영진에게 바라는 점]까지
    for col in df.columns[2:]:
        for j in range(df.shape[0]):
            try:
                # 고칠 게 있으면 고치고 기존 데이터 대체
                result_train = spell_checker.check(df[col][j])
                df[col][j] = result_train.as_dict()['checked']
            except:
                # 없으면 pass
                pass

In [54]:
clean_text(cons_df)

In [22]:
spelling(cons_df)

In [23]:
cons_df.to_excel(excel_writer = '건설업_전처리.xlsx')

In [6]:
# mecab 토큰화 및 불용어 처리
mecab = Mecab()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

korean_stopwords_path='./stopword_수정.txt'

# 텍스트 파일 오픈
with open(korean_stopwords_path,encoding='utf-8') as f:
    stopwords=f.readlines()
stopwords=[x.strip() for x in stopwords]

In [23]:
def tokenize(df):
    for col in df.columns[2:]:
        df[col+'토큰화'] = df[col].apply(mecab.morphs)
        df[col+'토큰화'] = df[col+'토큰화'].apply(lambda x: [item for item in x if item not in stopwords])

In [35]:
tokenize(cons_df)

In [36]:
cons_df.head()

,날짜,직무,요약,장점,단점,경영진에게 바라는 점,요약토큰화,장점토큰화,단점토큰화,경영진에게 바라는 점토큰화
회사,,,,,,,,,,
한국토지주택공사,2021. 03,서비스/고객지원,"""적당한 연봉과 자기 삶을 지킬 수 있는 균형 잡힌 회사""",공기업의 특성상 안정적인고 업무강도도 적당한 또한 사람들이 착함,공기업의 특성상 지방 또는 외곽쪽에서 근무하는 경우가 많음,외근의 경우억조 여자와 남자 모두에게 공평하면 좋겠습니다.,"["", 적당, 연봉, 자기, 삶, 지킬, 균형, 잡힌, 회사, ""]","[공기업, 특성, 상, 안정, 적, 인, 고, 업무, 강도, 적당, 사람, 착함]","[공기업, 특성, 상, 지방, 또는, 외곽, 쪽, 근무, 경우, 많, 음]","[외근, 경우, 억조, 여자, 남자, 모두, 공평, 면, 좋, 겠, .]"
한국토지주택공사,2021. 03,인사/총무,"""딱 일하려고 모인 사람들의 집단이었다""","휴가에 대한게 눈치보지 않고 자유로워서 좋았다 카페, 사내식당 등이 좋다",승진에 대한 경쟁을 하려는게 보인다. 윗사람 아랫사람 눈치 보임,터지는 문제들을 잘 해결하길. 앞으로 더 반성하길.,"["", 려고, 모인, 사람, 집단, 었, 다, ""]","[휴가, 대한, 게, 눈치보, 고, 자유, 로워서, 좋, 았, 다, 카페, ,, 사...","[승진, 대한, 경쟁, 려는, 게, 보인다, ., 윗사람, 아랫사람, 눈치, 보임]","[터지, 문제, 해결, 길, ., 앞, 더, 반성, 길, .]"
한국토지주택공사,2021. 03,인사/총무,"""안정적인듯 안정적이지 않은 조직. 기존 4,50대 직원들과 신입 20대 직원들의 ...","직원들을 위한 복지제도가 잘되어있음. 토목, 건축 전공자들에게는전공을 살릴수 있는 ...",순환근무로 인해서 짧으면 1년 길면 3년 사이에 계속 부서를 옮겨야한다. 부서 이동...,기본적인 마인드가 의전을 중요시 여기는 등 매우 수직적임. 시각을 좀 더 다양하게 ...,"["", 안정, 적, 인, 듯, 안정, 적, 조직, ., 기존, 4, ,, 50, 대...","[직원, 위한, 복지, 제도, 음, ., 토목, ,, 건축, 전공, 전공, 살릴, ...","[순환, 근무, 인해서, 짧, 으면, 1, 길, 면, 3, 사이, 계속, 부서, 옮...","[기본, 적, 인, 마인드, 의전, 중요시, 매우, 수직, 적, 임, ., 더, 다..."
한국토지주택공사,2021. 03,경영/기획/컨설팅,"""정말 좋은 회사였습니다 동료들도 너무 좋고 복지도 좋았어요""","연차 눈치 안보고 쓰기 가능, 분위기 자유로움, 동료들 좋음",발령날 때 전국구다보니 가족끼리 떨어져 사는 경우가 많이보였음,신입사원 많이 뽑아주시면 감사하겠습니다..인턴이었지만 정규직입사 원해요,"["", 정말, 좋, 회사, 였, 동료, 너무, 좋, 고, 복지, 좋, 았, 어요, ""]","[연차, 눈치, 안, 고, 쓰, 기, 가능, ,, 분위기, 자유, 로움, ,, 동료...","[발령, 날, 전국, 구다보, 니, 가족, 끼리, 떨어져, 사, 경우, 많이, 보였...","[신입, 사원, 많이, 뽑, 시, 면, 감사, 겠, ., ., 인턴, 었, 정규직,..."
한국토지주택공사,2021. 03,경영/기획/컨설팅,"""가족같은 분위기로 안정적으로 오래 다닐 수 있는 회사.""",안정적이다. 돈 꼬박꼬박 준다. 가족적이다. 가늘고 길게 다니고 싶은 사람에게 좋다.,진취적인 성향의 사람이라면 잘 맞지 않을 수 있습이다.,공기업이지만 성과에 따른 차등이 있으면 좋겠습니다. 의욕이 딸어져요.,"["", 가족, 분위기, 안정, 적, 오래, 다닐, 회사, ., ""]","[안정, 적, 다, ., 돈, 꼬박꼬박, 준다, ., 가족, 적, 다, ., 가늘,...","[진취, 적, 인, 성향, 사람, 라면, 맞, 습, 다, .]","[공기업, 성과, 따른, 차등, 으면, 좋, 겠, ., 욕, 딸, 져요, .]"


In [37]:
trade_df.to_excel(excel_writer = '유통_무역_운송_전처리.xlsx')
edu_df.to_excel(excel_writer = '교육업_전처리.xlsx')
cons_df.to_excel(excel_writer = '건설업_전처리.xlsx')